# Road Follower - Train Model

In this notebook we will train a neural network to take an input image, and output a set of x, y values corresponding to a target.

We will be using PyTorch deep learning framework to train ResNet18 neural network architecture model for road follower application.

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

### Download and extract data

Before you start, you should upload the ``road_following_<Date&Time>.zip`` file that you created in the ``data_collection.ipynb`` notebook on the robot. 

> If you're training on the JetBot you collected data on, you can skip this!

You should then extract this dataset by calling the command below:

In [15]:
!unzip -q road_following.zip

OSError: [Errno 12] Cannot allocate memory

You should see a folder named ``dataset_all`` appear in the file browser.

### Create Dataset Instance

Here we create a custom ``torch.utils.data.Dataset`` implementation, which implements the ``__len__`` and ``__getitem__`` functions.  This class
is responsible for loading images and parsing the x, y values from the image filenames.  Because we implement the ``torch.utils.data.Dataset`` class,
we can use all of the torch data utilities :)

We hard coded some transformations (like color jitter) into our dataset.  We made random horizontal flips optional (in case you want to follow a non-symmetric path, like a road
where we need to 'stay right').  If it doesn't matter whether your robot follows some convention, you could enable flips to augment the dataset.

In [2]:
def get_x(path, width):
    """Gets the x value from the image filename"""
    return (float(int(path.split("_")[1])) - width/2) / (width/2)

def get_y(path, height):
    """Gets the y value from the image filename"""
    return (float(int(path.split("_")[2])) - height/2) / (height/2)

class XYDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, random_hflips=False):
        self.directory = directory
        self.random_hflips = random_hflips
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
        
        all_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.image_paths = []
        for path in all_paths:
            try:
                with PIL.Image.open(path) as img:
                    img.verify()  # Check if image is readable
                self.image_paths.append(path)
            except Exception as e:
                print(f"Skipping invalid/corrupt image: {path} - {e}")
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = PIL.Image.open(image_path)
        width, height = image.size
        x = float(get_x(os.path.basename(image_path), width))
        y = float(get_y(os.path.basename(image_path), height))
      
        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)
            x = -x
        
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, torch.tensor([x, y]).float()
    
dataset = XYDataset('dataset_xy', random_hflips=False)

Skipping invalid/corrupt image: dataset_xy/xy_093_148_9e50747e-3538-11f0-8ad3-e0e1a91dc0f8.jpg - cannot identify image file 'dataset_xy/xy_093_148_9e50747e-3538-11f0-8ad3-e0e1a91dc0f8.jpg'
Skipping invalid/corrupt image: dataset_xy/xy_155_143_20b08a44-3539-11f0-82a2-e0e1a91dc0f8.jpg - cannot identify image file 'dataset_xy/xy_155_143_20b08a44-3539-11f0-82a2-e0e1a91dc0f8.jpg'
Skipping invalid/corrupt image: dataset_xy/xy_108_183_99213e70-3538-11f0-8ad3-e0e1a91dc0f8.jpg - cannot identify image file 'dataset_xy/xy_108_183_99213e70-3538-11f0-8ad3-e0e1a91dc0f8.jpg'
Skipping invalid/corrupt image: dataset_xy/xy_112_150_db5c4804-3536-11f0-b666-e0e1a91dc0f8.jpg - cannot identify image file 'dataset_xy/xy_112_150_db5c4804-3536-11f0-b666-e0e1a91dc0f8.jpg'
Skipping invalid/corrupt image: dataset_xy/xy_128_144_7f8c7ea0-353a-11f0-ac47-e0e1a91dc0f8.jpg - cannot identify image file 'dataset_xy/xy_128_144_7f8c7ea0-353a-11f0-ac47-e0e1a91dc0f8.jpg'
Skipping invalid/corrupt image: dataset_xy/xy_141_175_6

### Split dataset into train and test sets
Once we read dataset, we will split data set in train and test sets. In this example we split train and test a 90%-10%. The test set will be used to verify the accuracy of the model we train.

In [3]:
test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

### Create data loaders to load data in batches

We use ``DataLoader`` class to load data in batches, shuffle data and allow using multi-subprocesses. In this example we use batch size of 64. Batch size will be based on memory available with your GPU and it can impact accuracy of the model.

In [4]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

### Define Neural Network Model 

We use ResNet-18 model available on PyTorch TorchVision. 

In a process called transfer learning, we can repurpose a pre-trained model (trained on millions of images) for a new task that has possibly much less data available.


More details on ResNet-18 : https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

More Details on Transfer Learning: https://www.youtube.com/watch?v=yofjFQddwHE 

In [5]:
model = models.resnet18(pretrained=True)

ResNet model has fully connect (fc) final layer with 512 as ``in_features`` and we will be training for regression thus ``out_features`` as 1

Finally, we transfer our model for execution on the GPU

In [6]:
model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)

### Train Regression:

We train for 50 epochs and save best model if the loss is reduced. 

In [7]:
NUM_EPOCHS = 70
BEST_MODEL_PATH = 'best_steering_model_xy.pth'
best_loss = 1e9

optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += float(loss)
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += float(loss)
    test_loss /= len(test_loader)
    
    print('%f, %f' % (train_loss, test_loss))
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /media/nvidia/NVME/pytorch/pytorch-v1.9.0/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


1.099774, 5.333559
0.588573, 15.411853
0.114549, 0.392215
0.082219, 0.132861
0.064815, 0.093167
0.059095, 0.088969
0.049616, 0.041250
0.051873, 0.061752
0.055969, 0.053755
0.052253, 0.087857
0.037609, 0.021735
0.035707, 0.042695
0.047334, 0.042250
0.075989, 0.039624
0.051234, 0.027645
0.048626, 0.040291
0.027742, 0.097265
0.022839, 0.044729
0.024555, 0.037570
0.048993, 0.052573
0.043731, 0.033796
0.052157, 0.028080
0.032506, 0.056402
0.025617, 0.042925
0.015814, 0.084032
0.020236, 0.034118
0.038712, 0.042909
0.023673, 0.040444
0.033450, 0.173921
0.027493, 0.047875
0.024042, 0.049544
0.028454, 0.057552
0.030959, 0.023860
0.028496, 0.040926
0.019345, 0.037239
0.052163, 0.072557
0.042400, 0.039446
0.028579, 0.041119
0.012478, 0.049265
0.021038, 0.042176
0.015329, 0.043807
0.026978, 0.057919
0.037509, 0.064560
0.017938, 0.036735
0.026372, 0.073676
0.037456, 0.065856
0.018497, 0.078107
0.017290, 0.052700
0.029310, 0.067752
0.027818, 0.053409
0.016914, 0.033105
0.018783, 0.048073
0.010189, 0

Once the model is trained, it will generate ``best_steering_model_xy.pth`` file which you can use for inferencing in the live demo notebook.

If you trained on a different machine other than JetBot, you'll need to upload this to the JetBot to the ``road_following`` example folder.